# Responsible Pipeline Example

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import time
import joblib
import os

## Load data

In [ ]:
df = pd.read_csv('data/adult.csv', 
                 na_values=' ?', 
                 skipinitialspace=True)
df['income'] = df['income'].str.strip()
df = df.dropna(subset=['income', 'sex', 'race'])

In [ ]:
y = df['income'].map({'>50K': 1, '<=50K': 0})
df = df[~y.isna()]  
X = df.drop(['income'], axis=1, errors='ignore')
sensitive = df[['sex', 'race']]

In [ ]:
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

## Split data

In [ ]:
(X_train, X_test, y_train, y_test, 
 sens_train, sens_test) = train_test_split(
    X, y, sensitive, test_size=0.3, 
    stratify=y, random_state=42
)

## Define preprocessing and model pipeline

In [ ]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('clf', LogisticRegression(solver='liblinear'))
])

## CV and fairness audits

In [ ]:
skf = StratifiedKFold(n_splits=5)
metrics = {'accuracy': [], 'recall': [], 'precision': [], 'f1': []}
subgroup_performance = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    sens_val = sens_train.iloc[valid_idx]
    pipeline.fit(X_tr, y_tr)
    y_pred = pipeline.predict(X_val)
    metrics['accuracy'].append(accuracy_score(y_val, y_pred))
    metrics['recall'].append(recall_score(y_val, y_pred))
    metrics['precision'].append(precision_score(y_val, y_pred))
    metrics['f1'].append(f1_score(y_val, y_pred))

    # -- Attribute-level audits --
    for attr in sens_val.columns:
        for grp in sens_val[attr].unique():
            mask = sens_val[attr] == grp
            if mask.sum() == 0:
                continue
            subgroup_performance.append({
                'type': 'attribute',
                'attribute': attr,
                'group': grp,
                'accuracy': accuracy_score(y_val[mask], y_pred[mask])
            })
    # -- Intersectional audits --
    intersection_series = sens_val['sex'].astype(str) + "_" + sens_val['race'].astype(str)
    for grp in intersection_series.unique():
        mask = intersection_series == grp
        if mask.sum() == 0:
            continue
        subgroup_performance.append({
            'type': 'intersectional',
            'attribute': 'sex_race',
            'group': grp,
            'accuracy': accuracy_score(y_val[mask], y_pred[mask])
        })

## Fit the model and evaluate

In [ ]:
pipeline.fit(X_train, y_train)
y_test_pred = pipeline.predict(X_test)
overall = {
    'accuracy': accuracy_score(y_test, y_test_pred),
    'recall': recall_score(y_test, y_test_pred),
    'precision': precision_score(y_test, y_test_pred),
    'f1': f1_score(y_test, y_test_pred)
}
print("Overall test metrics:", overall)

## Test subgroup accuracy

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
sens_test_combined = sens_test['sex'].astype(str) + "_" + sens_test['race'].astype(str)

groups = []
accuracies = []
f1s = []
precisions = []
recalls = []

for grp in sens_test_combined.unique():
    mask = sens_test_combined == grp
    acc = accuracy_score(y_test[mask], y_test_pred[mask])
    f1 = f1_score(y_test[mask], y_test_pred[mask])
    precision = precision_score(y_test[mask], y_test_pred[mask])
    recall = recall_score(y_test[mask], y_test_pred[mask])
    groups.append(grp)
    accuracies.append(acc)
    f1s.append(f1)
    precisions.append(precision)
    recalls.append(recall)

In [ ]:
df_metrics = pd.DataFrame({
    'Group': groups,
    'Accuracy': accuracies,
    'F1': f1s,
    'Precision': precisions,
    'Recall': recalls
})
df_metrics = df_metrics.set_index('Group')
df_metrics

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_group_metrics_bar(
    df,
    metrics,
    palette_name="Greys",
    sns_theme=None,
    legend_pos=None,
    legend_args=None,
    label_args=None,
    x_tick_rotation=90,
    figsize=(12, 6),
    show_legend=True,
    **kwargs
):
    if sns_theme is not None:
        sns.set_theme(style=sns_theme)
    else:
        sns.set_theme()
    palette = sns.color_palette(palette_name, n_colors=len(metrics))
    x = np.arange(len(df))
    width = 0.8 / len(metrics)

    fig, ax = plt.subplots(figsize=figsize)
    for i, metric in enumerate(metrics):
        ax.bar(x + i * width, df[metric], width, label=metric, color=palette[i])

    ax.set_xticks(x + width * (len(metrics) - 1) / 2)
    ax.set_xticklabels(df.index, rotation=x_tick_rotation, fontsize=kwargs.get("xtick_fontsize", 12))

    if label_args is not None:
        ax.set_ylabel(**label_args)
    else:
        ax.set_ylabel(kwargs.get("ylabel", "Score"))

    ax.set_title(kwargs.get("title", "Model Performance Metrics by Group"))

    if show_legend:
        if legend_args is not None:
            if legend_pos is not None:
                ax.legend(loc=legend_pos, **legend_args)
            else:
                ax.legend(**legend_args)
        else:
            ax.legend()

    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_group_metrics_bar(
    df_metrics, ['Accuracy', 'F1', 
                'Precision', 
                'Recall'],
    sns_theme="whitegrid",
    show_legend=True,
    
    label_args={'ylabel': 'Score', 'fontsize': 12}
    )


## SHAP Explainability

In [ ]:
import shap
X_sample = X_train.sample(n=100, random_state=1)
X_sample_proc = pipeline.named_steps['preprocessing'].transform(X_sample)
explainer = shap.Explainer(pipeline.named_steps['clf'], X_sample_proc)
shap_values = explainer(pipeline.named_steps['preprocessing'].transform(X_test[:100]))
shap.summary_plot(shap_values, feature_names=pipeline.named_steps['preprocessing'].get_feature_names_out(), show=False)

## Flag uncertain predictions for human review

In [ ]:
probs = pipeline.predict_proba(X_test)[:, 1]
edge_cases = np.where((probs > 0.45) & (probs < 0.55))[0]
X_test.iloc[edge_cases]

In [ ]:
print("Edge cases needing manual review:", len(edge_cases))

## Documentation and audit trail

In [ ]:
metadata = {
    'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
    'features': list(X.columns),
    'sensitive_features': list(sensitive.columns),
    'model_params': pipeline.named_steps['clf'].get_params(),
    'cv_metrics': metrics,
    'overall_metrics': overall,
    'subgroup_performance': subgroup_performance
}
print(metadata)
os.makedirs('audit', exist_ok=True)
joblib.dump({'pipeline': pipeline, 
             'metadata': metadata}, 
             'audit/responsible_model_audit.joblib')